In the setup, we create a SQLAlchemy session, define a Parent model 
subclassing `FunctionRelator`, and a Function model subclassing 
`FunctionMixin`.

In [1]:
from sqlalchemy_function import FunctionMixin, FunctionRelator

# standard session creation
from sqlalchemy import create_engine, Column, ForeignKey, Integer
from sqlalchemy.orm import relationship, sessionmaker, scoped_session
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite:///:memory:')
session_factory = sessionmaker(bind=engine)
Session = scoped_session(session_factory)
session = Session()
Base = declarative_base()

# subclass `FunctionRelator` for models with a relationship to Function models
class Parent(FunctionRelator, Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)
    functions = relationship('Function', backref='parent')

# subclass `FunctionMixin` to define a Function model
class Function(FunctionMixin, Base):
    __tablename__ = 'function'
    id = Column(Integer, primary_key=True)
    parent_id = Column(Integer, ForeignKey('parent.id'))

Base.metadata.create_all(engine)

In [2]:
def foo(*args, **kwargs):
    print('My args are', args)
    print('My kwargs are', kwargs)
    return 'return value'

In [3]:
func = Function(foo, 'hello world', goodbye='moon')
func()

My args are ('hello world',)
My kwargs are {'goodbye': 'moon'}


'return value'

In [4]:
parent = Parent()
parent.functions = foo
print(parent.functions)
parent.functions[0]()

My args are ()
My kwargs are {}


'return value'

In [5]:
def bar():
    return 'bar return value'

parent.functions = [foo, bar]
print(parent.functions)
[f() for f in parent.functions]

[<__main__.Function object at 0x7f0e1160cd30>, <__main__.Function object at 0x7f0e1160cdd8>]
My args are ()
My kwargs are {}


['return value', 'bar return value']